In [ ]:
#import required libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup
from difflib import SequenceMatcher


In [ ]:
#request html content, both for regular season and playoffs
url_base_reg = 'https://www.basketball-reference.com/leagues/NBA_{}.html'
url_base_play = 'https://www.basketball-reference.com/playoffs/NBA_{}.html'
url_base_payroll = 'https://hoopshype.com/salaries/{}/'

years = [year for year in range(2013, 2023)]

teams = ["Boston Celtics", "Brooklyn Nets", "New York Knicks", "Philadelphia 76ers", "Toronto Raptors", "Chicago Bulls",
"Cleveland Cavaliers", "Detroit Pistons", "Indiana Pacers", "Milwaukee Bucks", "Atlanta Hawks", "Charlotte Hornets",
"Miami Heat", "Orlando Magic", "Washington Wizards", "Denver Nuggets", "Minnesota Timberwolves", "Oklahoma City Thunder",
"Portland Trail Blazers", "Utah Jazz", "Golden State Warriors", "Los Angeles Clippers", "Los Angeles Lakers", "Phoenix Suns",
"Sacramento Kings", "Dallas Mavericks", "Houston Rockets", "Memphis Grizzlies", "New Orleans Pelicans", "San Antonio Spurs", "Charlotte Bobcats", "New Orleans Hornets"]

df_main = pd.DataFrame()

cpi = pd.read_csv('data/cpi-u_2013-22.csv')
bri = pd.read_csv('data/total_league_revenues.csv')

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [ ]:
for i in years:

    url_reg = url_base_reg.format(i)

    per_game = requests.get(url_reg)

    #create BeautifulSoup object
    soup = BeautifulSoup(per_game.content, 'html.parser')

    #locate correct table
    table = soup.find(lambda tag: tag.name=='table' and tag.has_attr('id') and tag['id']=="per_game-team") 
    rows = table.findAll(lambda tag: tag.name=='tr')

    #create DataFrame
    df = pd.read_html(str(table))[0]

    #clean DataFrame
    df = df.drop(index=30)
    df['Team'] = df['Team'].str.replace('*', '')
    df.insert(2, "Year", i, True)
    df = df.drop(columns=['Rk', 'G', 'MP'])


    #add advanced stats to DataFrame
    table = soup.find(lambda tag: tag.name=='table' and tag.has_attr('id') and tag['id']=="advanced-team") 
    rows = table.findAll(lambda tag: tag.name=='tr')
    df_adv = pd.read_html(str(table))[0]
    
    df_adv.columns = df_adv.columns.droplevel()
    df_adv = df_adv.drop(index=30)
    df_adv['Team'] = df_adv['Team'].str.replace('*', '')
    df_adv = df_adv.drop(columns=["Rk", "L", "PW", "PL", "Unnamed: 17_level_1", "Unnamed: 22_level_1", "Unnamed: 27_level_1", "Arena"])


    #add playoff wins to DataFrame
    url_play = url_base_play.format(i)
    playoff_wins = requests.get(url_play)
    soup = BeautifulSoup(playoff_wins.content, 'html.parser')
    table = soup.find(lambda tag: tag.name=='table' and tag.has_attr('id') and tag['id']=="advanced-team")
    df_play = pd.read_html(str(table))[0]
    
    df_play.columns = df_play.columns.droplevel()
    df_play = df_play.drop(index=16)
    df_play = df_play.rename(columns={"Tm": "Team", "W" : "Playoff_W"})
    df_play = df_play[["Team", "Playoff_W"]]


    #add payroll data
    url_payroll = url_base_payroll.format(f'{i-1}-{i}')
    payrolls = requests.get(url_payroll)
    soup = BeautifulSoup(payrolls.content, 'html.parser')
    table = soup.find("table", class_="hh-salaries-ranking-table hh-salaries-table-sortable responsive")
    rows = table.findAll(lambda tag: tag.name=='tr')
    df_payroll = pd.read_html(str(table))[0]

    df_payroll = df_payroll.iloc[:, [1,2]]
    df_payroll[f'{i-1}/{i-2000}'] = df_payroll[f'{i-1}/{i-2000}'].str.replace('[$,]', '').astype(int)
    df_payroll = df_payroll.rename(columns={f'{i-1}/{i-2000}': 'Payroll'})

    team_replace = {row: team for row in df_payroll['Team'].to_list() for team in teams if similar(row, team)>=0.53}

    for index, team in team_replace.items():
        df_payroll.loc[df_payroll['Team'] == index, 'Team'] = team
            
    # adjust payroll for inflation, basketball-related-income
    df_payroll['Adjusted Payroll'] = df_payroll['Payroll'] / (cpi[cpi['Year'] == (i - 1)]['Annual'].values[0] 
    + bri[bri['Year'] == (i - 1)]['Revenue'].values[0])


    # merge all datasets
    df = pd.merge(df, df_adv, on='Team')
    df = pd.merge(df, df_play, on='Team')
    df = pd.merge(df, df_payroll, on='Team')

    df_main = pd.concat([df_main, df])
    df_main = df_main.reset_index(drop=True)


In [ ]:
df_main

In [ ]:
#write data to csv
df_main.to_csv('data/playoff_per_game_2013-22.csv')